# 3. Text preprocessing

## Setup

In [1]:
%run __init__.py

INFO:root:Starting logger


In [2]:
import pandas as pd

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Common pipeline

## COVID-19
bla bla bla

### Loading the dataframe

In [ ]:
CORD_DATASET_DIR = os.path.join(DATA_DIR, 'cord19')
CORD19_FILE_PATH = os.path.join(CORD_DATASET_DIR, 'cord19_dataframe.pkl')

cord19_df = pd.read_pickle(CORD19_FILE_PATH)

## Agriculture

### Loading the dataframe

In [3]:
AGRICULTURE_DATASET_DIR = os.path.join(DATA_DIR, 'agriculture')
PMC_FILE_PATH = os.path.join(AGRICULTURE_DATASET_DIR, 'pmc_dataframe.pkl')

pmc_df = pd.read_pickle(PMC_FILE_PATH)

### Preprocessing text

In [4]:
publications = pmc_df['text_cleaned'].values

In [5]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from sklearn.base import TransformerMixin, BaseEstimator

nlp = en_core_web_sm.load()
nlp.Defaults.stop_words |= {"et","al", "introduction", "Fig", "fig", "figure"}

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, n_jobs=1):
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *args, **kwargs):
        return [self._preprocess_text(text) for text in X]

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        doc = nlp(text)
        return [t.lemma_ for t in doc if len(t.text) > 2 and
                not t.is_stop and t.text not in string.punctuation
                and t.is_alpha and not t.is_digit]

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]
    
    def _remove_digits(self, doc):
        return [t for t in doc if t.is_alpha and not t.is_digit]

    def _lemmatize(self, doc):
        return [t.lemma_ for t in doc if len(t.text) > 2]
    

### LDA

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
    return doc

lda_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                     ('vectorizer', CountVectorizer(preprocessor=dummy, tokenizer=dummy)),
                     ('model', LatentDirichletAllocation())])


In [7]:
from sklearn.model_selection import RandomizedSearchCV

search_params = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'model__n_components': [10, 15, 20, 25, 30],
    'model__learning_decay': [.5, .7, .9]
}

search = RandomizedSearchCV(lda_pipeline, param_distributions=search_params, n_iter=15)
search.fit(publications)

KeyboardInterrupt: 

In [71]:
best_lda_model = search.best_estimator_

print(f"Best pipeline parameters: {search.best_params_}")
print(f"Best Log Likelihood Score: {search.best_score_}")

Best pipeline parameters: {'vectorizer__ngram_range': (1, 1), 'model__n_components': 10, 'model__learning_decay': 0.7}
Best Log Likelihood Score: -847826.416759485


In [14]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



Visualization. Do this with the best model obtained before.

In [28]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

In [25]:
# {'vectorizer__ngram_range': (1, 1), 'model__n_components': 10, 'model__learning_decay': 0.7}

preprocessed_text = TextPreprocessor().fit_transform(publications)
tf_vectorizer = CountVectorizer(preprocessor=dummy, tokenizer=dummy, ngram_range=(1, 1))
dtm_tf = tf_vectorizer.fit_transform(preprocessed_text)
lda_tf = LatentDirichletAllocation(n_components=10, learning_decay=0.7).fit(dtm_tf)

In [26]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_tf, tf_feature_names, 10)

Topic #0: habitat plant growth grassland cold bird accession Arabidopsis regime high
Topic #1: plant virus infect infection non sample infected specie leave light
Topic #2: plant protein expression gene root level response acid stress induce
Topic #3: system food crop plant practice take farmer orally decoction farm
Topic #4: soil rice study site high value water sample season emission
Topic #5: plant population value datum volatile high result base flower metabolite
Topic #6: soil crop root study plant rodent day yield Salmonella stage
Topic #7: plant gene stress transgenic expression line control show protein salt
Topic #8: plant soil rice increase root treatment stress high concentration content
Topic #9: plant application crop MED base sensor treatment report system node



In [29]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  18.759195
7       0.648955  71.707100       2        1  16.296523
8      71.019684   9.971006       3        1  14.985811
4       3.785981  33.185135       4        1   9.804348
3     -42.666500  -9.222070       5        1   9.751730
5      -3.621689 -43.526516       6        1   7.538485
9      -3.632402  -4.171708       7        1   7.538081
1     -36.584400  35.804413       8        1   6.803786
6      44.391380  50.615383       9        1   4.462845
0      40.235809 -31.591665      10        1   4.059195, topic_info=            Term          Freq         Total Category  logprob  loglift
23617      plant  13700.000000  13700.000000  Default  30.0000  30.0000
18557       gene   2536.000000   2536.000000  Default  29.0000  29.0000
26083       soil   2413.000000   2413.000000  Default  28.0000  28.0000
25186       rice   1955.000000   1955.000000  Default  27.0000  27.0000
26512     stress   2729.000000   2729.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
25849       show     81.472336   1825.499415  Topic10  -5.5208   0.0948
26560      study     73.404443   2115.058055  Topic10  -5.6251  -0.1567
15594  condition     65.436632   1471.114715  Topic10  -5.7400   0.0915
20691      level     64.889947   2138.843780  Topic10  -5.7484  -0.2911
28170      value     64.849132   1225.942444  Topic10  -5.7490   0.2648

[764 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
11         1  0.332631   ABA
11         2  0.655183   ABA
11         3  0.003360   ABA
11         6  0.006720   ABA
69         1  0.918225  ALDH
...      ...       ...   ...
28708      1  0.034728  zinc
28708      2  0.006946  zinc
28708      3  0.027782  zinc
28708      6  0.902930  zinc
28708      8  0.027782  zinc

[2316 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 9, 5, 4, 6, 10, 2, 7, 1])

### Latent Semantic Analysis (LSA)

In [16]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

lsa_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                 ('vectorizer', TfidfVectorizer(preprocessor=dummy, tokenizer=dummy)),
                 ('model', TruncatedSVD(n_components=10))])

In [21]:
tf_feature_names = lsa_pipeline['vectorizer'].get_feature_names()
print_top_words(lsa_pipeline['model'], tf_feature_names, 10)

Topic #0: plant stress gene rice soil expression transgenic root increase protein
Topic #1: rice soil crop season emission residue system farm farmer winter
Topic #2: food farmer farm system household farming livestock crop climate diversity
Topic #3: stress heat salt tolerance drought NaCl food ROS salinity ALDH
Topic #4: transgenic gene rice expression emission crop line season protein drought
Topic #5: medicinal salt heat yield plant stress informant NaCl hypertension season
Topic #6: transgenic salt rice sensor food NaCl iot emission node system
Topic #7: emission sensor rice node season iot temperature heat defense Las
Topic #8: medicinal gene mutant informant hypertension wild decoction hydrophilin protein RNAi
Topic #9: endophyte yield sensor node grain root iot bean soil irrigation



In [30]:
lsa_search_params = {
    'vectorizer__min_df': [1, 2, 3],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2), (1, 3), (2, 3)],
    'model__n_components': [5, 10, 15, 20, 25, 30],
    'model__algorithm': ["arpack", "randomized"]
}

lsa_search = RandomizedSearchCV(lsa_pipeline, param_distributions=lsa_search_params, n_iter=2)
lsa_search.fit(publications)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('preprocessing', TextPreprocessor()),
                ('vectorizer',
                 TfidfVectorizer(preprocessor=<function dummy at 0x7f01195fd7b8>,
                                 tokenizer=<function dummy at 0x7f01195fd7b8>)),
                ('model', TruncatedSVD(n_components=10))]) does not.

Visualization

In [ ]:
tf_feature_names = lsa_pipeline['vectorizer'].get_feature_names()
print_top_words(lsa_pipeline['model'], tf_feature_names, 10)

### NMF

In [19]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

nmf_pipeline = Pipeline([('preprocessing', TextPreprocessor()),
                     ('vectorizer', TfidfVectorizer(preprocessor=dummy, tokenizer=dummy)),
                     ('model', NMF(n_components=10))])

In [ ]:
nmf_pipeline.fit_transform(publications)

In [23]:
tf_feature_names = nmf_pipeline['vectorizer'].get_feature_names()
print_top_words(nmf_pipeline['model'], tf_feature_names, 10)

Topic #0: plant endophyte defense herbivore metabolite volatile GSL insect touch expression
Topic #1: rice soil emission season crop residue paddy straw winter treatment
Topic #2: food farmer farm system household CSA climate farming crop livestock
Topic #3: stress plant salt heat tolerance NaCl drought gene expression treatment
Topic #4: transgenic plant gene expression line protein sequence transcript overexpressor RNAi
Topic #5: metal toxicity soil plant root concentration heavy acid uptake increase
Topic #6: sensor iot node IoT wireless network communication Smart power Campus
Topic #7: plant infect CMV Las virus infected infection NahG citri symptomatic
Topic #8: medicinal plant informant hypertension decoction Herb traditional specie orally ailment
Topic #9: crop yield soil legume grain bird bean habitat residue wheat



In [ ]:
nmf_pip